## Question 1. Backpropagation

In [51]:
import numpy as np
import gzip
import idx2numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from operator import add
from scipy.special import expit
import math
from tqdm import tqdm
import joblib
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

In [2]:
training_loss = {
    "tanh" : [],
    "sigmoid": [],
    "relu": [],
    "linear": []
}

validation_loss = {
    "tanh" : [],
    "sigmoid": [],
    "relu": [],
    "linear": []
}

In [3]:
#helper function to save the learnt weights and biases.
def dump_learned_weights_and_bias(weights, bias, activation, epoch):
    weights_path = "checkpoints/"+activation+"/"+str(epoch)+"_"+activation+"_"+"weights.pkl"
    bias_path = "checkpoints/"+activation+"/"+str(epoch)+"_"+activation+"_"+"bias.pkl"
    joblib.dump(weights, weights_path)
    joblib.dump(bias, bias_path)
    return

##### **Question 1: Part 1 to Part 4**

In [4]:
class MyNeuralNetwork():
    """
    My implementation of a Neural Network Classifier.
    """

    acti_fns = ['relu', 'sigmoid', 'linear', 'tanh', 'softmax']
    weight_inits = ['zero', 'random', 'normal']

    def __init__(self, n_layers, layer_sizes, activation, learning_rate, weight_init, batch_size, num_epochs):
        """
        Parameters
        ----------
        n_layers : int value specifying the number of layers
        layer_sizes : integer array of size n_layers specifying the number of nodes in each layer
        activation : string specifying the activation function out of relu, sigmoid, linear, tanh
        learning_rate : float value specifying the learning rate to be used
        weight_init : string specifying the weight initialization function to be used: zero, random, normal
        batch_size : int value specifying the batch size to be used
        num_epochs : int value specifying the number of epochs to be used
        """

        if activation not in self.acti_fns:
            raise Exception('Incorrect Activation Function')

        if weight_init not in self.weight_inits:
            raise Exception('Incorrect Weight Initialization Function')
        
        l = [784] #append input and output layer to existing layers
        l.extend(layer_sizes)
        l.append(10)
        
        np.random.seed(420)
        self.n_layers = n_layers
        self.layer_sizes = l
        self.activation = activation
        self.learning_rate = learning_rate
        self.weight_init = weight_init
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.weights = {}
        self.bias = {}
        self.cache = {} #cache
        self.initialise_weights()   #initialise model weights and biases

    def compare_with_mlp(self,loss_from_your_model,X_train,y_train):
        # comparing with MLP
        # X is all data and Y is all labels
        act = self.activation
        if(act=="sigmoid"):
            act = "logistic"
        if(act=="linear"):
            act = "identity"
        mlp = MLPClassifier(hidden_layer_sizes=(256, 128, 64), activation =act ,solver = 'sgd' , alpha = 0, batch_size = 32, max_iter=50, learning_rate_init = self.learning_rate, learning_rate = 'constant', shuffle=True, momentum = 0, nesterovs_momentum=False, validation_fraction = 0.1, verbose=True, warm_start=True)
        mlp.fit(X_train, y_train)
        mlp_file = "checkpoints/sklearn/"+"50_"+self.activation+"_model.pkl"
        joblib.dump(mlp, mlp_file)
        mlp.fit(X_train, y_train)
        mlp_file = "checkpoints/sklearn/"+"100_"+self.activation+"_model.pkl"
        joblib.dump(mlp, mlp_file)
        loss_from_sklearn = mlp.loss_curve_
        sns.set()
        plt.plot(loss_from_sklearn,label="sklearn")
        plt.plot(loss_from_your_model,label="my NN")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend(loc="upper left")
        plt.savefig(self.activation+"_comparison.png")
        plt.close()

    def relu(self, X):
        """
        Calculating the ReLU activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        x_calc = X * (X > 0)
        return x_calc

    def relu_grad(self, X):
        """
        Calculating the gradient of ReLU activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        x_calc = np.where(X < 0, 0, 1)
        return x_calc

    def sigmoid(self, X):
        """
        Calculating the Sigmoid activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        x_calc = expit(X)
        return x_calc

    def sigmoid_grad(self, X):
        """
        Calculating the gradient of Sigmoid activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        sig = self.sigmoid(X)
        x_calc = sig * (1 - sig)
        return x_calc

    def linear(self, X):
        """
        Calculating the Linear activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        x_calc = X
        return x_calc

    def linear_grad(self, X):
        """
        Calculating the gradient of Linear activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        x_calc = np.ones(X.shape)
        return x_calc

    def tanh(self, X):
        """
        Calculating the Tanh activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        x_calc = np.tanh(X)
        return x_calc

    def tanh_grad(self, X):
        """
        Calculating the gradient of Tanh activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        tn = self.tanh(X)
        x_calc = 1 - tn**2
        return x_calc

    def softmax(self, X):
        """
        Calculating the ReLU activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        X_ = X - np.max(X)
        X_ = np.exp(X_)
        x_calc = X_ / np.sum(X_, axis=0)
        return x_calc

    def softmax_grad(self, X):
        """
        Calculating the gradient of Softmax activation for a particular layer

        Parameters
        ----------
        X : 1-dimentional numpy array 

        Returns
        -------
        x_calc : 1-dimensional numpy array after calculating the necessary function over X
        """
        #calculate gradient of softmax
        z = self.softmax(X)[None, :] # (1,n)
        ex1 = np.einsum('ij,ik->ijk', z, z) # (1,n,n) [z1^2, z2*z1, z3*z1 ...]
        ex2 = np.diag(z[0]) # (n,n) [z1, 0, 0 ...]
        x_calc = ex2 - ex1 # (n,n) derivative of softmax
        return x_calc
    
    def initialise_weights(self):
        """
        Calculating the initial weights for all the layers

        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        for i in range (1, len(self.layer_sizes)):
            self.bias['b'+str(i)] = self.zero_init((self.layer_sizes[i], 1)) #initialise bias with 0s
            if self.weight_init == 'zero':
                self.weights['W'+str(i)] = self.zero_init((self.layer_sizes[i], self.layer_sizes[i-1]))
                
            elif self.weight_init == 'random':
                self.weights['W'+str(i)] = self.random_init((self.layer_sizes[i], self.layer_sizes[i-1]))
                
            elif self.weight_init == 'normal':
                self.weights['W'+str(i)] = self.normal_init((self.layer_sizes[i], self.layer_sizes[i-1]))

    def return_activation_function(self, activation):
        activation_function = None
        activation_gradient = None
        if activation=="relu":
            activation_function = self.relu
            activation_gradient = self.relu_grad
        elif activation=="sigmoid":
            activation_function = self.sigmoid
            activation_gradient = self.sigmoid_grad
        elif activation=="linear":
            activation_function = self.linear
            activation_gradient = self.linear_grad
        elif activation=="tanh":
            activation_function = self.tanh
            activation_gradient = self.tanh_grad
        elif activation=="softmax":
            activation_function = self.softmax
            activation_gradient = self.softmax_grad
        else:
            print("Invalid activation function")
            return None, None
        return activation_function, activation_gradient

    def zero_init(self, shape):
        """
        Calculating the initial weights after Zero Activation for a particular layer

        Parameters
        ----------
        shape : tuple specifying the shape of the layer for which weights have to be generated 

        Returns
        -------
        weight : 2-dimensional numpy array which contains the initial weights for the requested layer
        """
        weight = np.zeros(shape)
        return weight

    def random_init(self, shape):
        """
        Calculating the initial weights after Random Activation for a particular layer

        Parameters
        ----------
        shape : tuple specifying the shape of the layer for which weights have to be generated 

        Returns
        -------
        weight : 2-dimensional numpy array which contains the initial weights for the requested layer
        """
        weight = np.random.rand(shape[0], shape[1]) * 0.01 
        return weight

    def normal_init(self, shape):
        """
        Calculating the initial weights after Normal(0,1) Activation for a particular layer

        Parameters
        ----------
        shape : tuple specifying the shape of the layer for which weights have to be generated 

        Returns
        -------
        weight : 2-dimensional numpy array which contains the initial weights for the requested layer
        """
        weight = np.random.randn(shape[0], shape[1]) * 0.01 #normal distribution with mean 0 and standard deviation 1
        return weight
    
    def one_hot_encode(self, y, n_labels):
        mat = np.eye(n_labels)[y]
        return mat

    def sg_descent(self):
        for i in range(1,self.n_layers+2):
            self.weights['W'+str(i)] -= self.gradients['dW'+str(i)] * self.learning_rate
            self.bias['b'+str(i)] -= self.gradients['db'+str(i)] * self.learning_rate
    
    def cross_entropy_loss(self, y, y_pred):
        return -np.sum(np.multiply(y.T, np.log(y_pred+1e-8)))/y.shape[0]

    def forward_propagation(self, X):
        activation, _ = self.return_activation_function(self.activation)
        self.cache['A'+str(0)] = X.T
        for i in range(1, self.n_layers+2):
            if i==1:
                self.cache['Z'+str(i)]= np.matmul(self.weights['W'+str(i)], self.cache['A'+str(0)]) + self.bias['b'+str(i)]
            else:
                self.cache['Z'+str(i)]= np.matmul(self.weights['W'+str(i)], self.cache['A'+str(i-1)]) + self.bias['b'+str(i)]
            if i==self.n_layers+1:
                self.cache['A'+str(i)] = self.softmax(self.cache['Z'+str(i)])
            else:
                self.cache['A'+str(i)] = activation(self.cache['Z'+str(i)]) 
        return self.cache['A'+str(self.n_layers+1)]

    def backward_propagation(self, y, output):
        self.gradients = {}
        _, gradient_activation = self.return_activation_function(self.activation)
        dZ = output - y.T
        for i in range(len(self.layer_sizes)-1, 0, -1):
            # print(dZ.shape, self.cache['A'+str(i-1)].shape)
            self.gradients['dW'+str(i)] = np.matmul(dZ, self.cache['A'+str(i-1)].T) * 1/y.shape[0]
            self.gradients['db'+str(i)] = np.sum(dZ, axis=1, keepdims=True) * 1/y.shape[0] 
            if i>1:
                dA = np.matmul(self.weights['W'+str(i)].T, dZ)
                dZ = dA * gradient_activation(self.cache['Z'+str(i-1)])
        return self.gradients

    def fit(self, X, y):
        """
        Fitting (training) the linear model.

        Parameters
        ----------
        X : 2-dimensional numpy array of shape (n_samples, n_features) which acts as training data.

        y : 2-dimensional numpy array one hot encoded array corresponding to the training labels.
        
        Returns
        -------
        self : an instance of self
        """
        global training_loss
        global validation_loss
        for epoch in tqdm(range(self.num_epochs)):
            print("\n----------- Epoch:", epoch+1, "of", self.num_epochs, "-----------")
            #divide dataset into training and validation
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.11, shuffle=True) #random shuffle
            n_batches = int(np.ceil(X_train.shape[0]/self.batch_size))
            for i in range(n_batches):
                # print("Training Batch:", i)
                ind1 = i*self.batch_size
                ind2 = np.min([ind1+self.batch_size, X_train.shape[0]-1])
                X_train_batch = X_train[ind1:ind2]
                y_train_batch = y_train[ind1:ind2]

                #forward propagation
                a_n = self.forward_propagation(X_train_batch)
                #back propagation and update weights
                self.backward_propagation(y_train_batch, a_n)
                self.sg_descent()
            
            if epoch==49 or epoch==99 or epoch==2:
                dump_learned_weights_and_bias(self.weights, self.bias, self.activation, epoch+1)                

            t_loss = self.cross_entropy_loss(y_train, self.predict_proba(X_train))
            v_loss = self.cross_entropy_loss(y_val, self.predict_proba(X_val))
            training_loss[self.activation].append(t_loss) 
            validation_loss[self.activation].append(v_loss)

            print("\nTraining Results for epoch:", epoch+1, "of", self.num_epochs)
            print("Training Loss:", t_loss)
            print("Training Accuracy:", self.score(X_train, np.argmax(y_train, axis=-1))*100)
            print("Validation Results for epoch:", epoch+1, "of", self.num_epochs)
            print("Validation Loss:", v_loss)
            print("Validation Accuracy:", self.score(X_val, np.argmax(y_val, axis=-1))*100)

        return self

    def predict_proba(self, X):
        """
        Predicting probabilities using the trained linear model.

        Parameters
        ----------
        X : 2-dimensional numpy array of shape (n_samples, n_features) which acts as testing data.

        Returns
        -------
        y : 2-dimensional numpy array of shape (n_samples, n_classes) which contains the 
            class wise prediction probabilities.
        """

        # return the numpy array y which contains the predicted values
        return self.forward_propagation(X)

    def predict(self, X):
        """
        Predicting values using the trained linear model.

        Parameters
        ----------
        X : 2-dimensional numpy array of shape (n_samples, n_features) which acts as testing data.

        Returns
        -------
        y : 1-dimensional numpy array of shape (n_samples,) which contains the predicted values.
        """
        y_pred = self.forward_propagation(X)
        return np.argmax(y_pred, axis=0)

    def score(self, X, y):
        """
        Predicting values using the trained linear model.

        Parameters
        ----------
        X : 2-dimensional numpy array of shape (n_samples, n_features) which acts as testing data.

        y : 1-dimensional numpy array of shape (n_samples,) which acts as testing labels.

        Returns
        -------
        acc : float value specifying the accuracy of the model on the provided testing set
        """
        y_pred = self.forward_propagation(X)
        acc = np.mean(y == np.argmax(y_pred.T, axis=-1))
        return acc



In [5]:
def flatten(dataset):
  nimages, nfeatures = dataset.shape
  lis = np.linspace(1, nfeatures, nfeatures, dtype=np.int64)
  lis = lis.tolist()
  lis = [str(x) for x in lis]
  dataset_flat = dataset.reshape((nimages, nfeatures))
  dataset_df = pd.DataFrame(dataset_flat, columns=lis)
  return dataset_df

In [6]:
images = gzip.open('fashion-mnist_train/FashionMNIST/raw/train-images-idx3-ubyte.gz','r')
labels = gzip.open('fashion-mnist_train/FashionMNIST/raw/train-labels-idx1-ubyte.gz','r')
mnist_images = idx2numpy.convert_from_file(images)
mnist_labels = idx2numpy.convert_from_file(labels)
#split into 90:10 train:test
mnist_train, mnist_test, mnist_train_labels, mnist_test_labels = train_test_split(mnist_images, mnist_labels, test_size=0.1, random_state=42, shuffle=True)

In [7]:
scaler = StandardScaler()
scaler.fit(mnist_images.reshape(mnist_images.shape[0], -1))
# mnist_test = scaler.transform(mnist_test.reshape(mnist_test.shape[0], -1))
# mnist_train = scaler.transform(mnist_train.reshape(mnist_train.shape[0], -1))
mnist_test_flat = flatten(mnist_test.reshape(mnist_test.shape[0], -1))/255.0
mnist_train_flat = flatten(mnist_train.reshape(mnist_train.shape[0], -1))/255.0

In [8]:
all_mnist_train = mnist_images.reshape(mnist_images.shape[0], -1)/255.0

In [9]:
mnist_train_flat.head()

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.015686,0.003922,0.003922,0.0,0.321569,0.705882,0.305882,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


##### **Question 1: Part 5**


In [11]:
#Training using ReLU
nn1 = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="relu")
nn1.fit(np.array(mnist_train_flat, dtype='float32'), nn1.one_hot_encode(mnist_train_labels.astype(int),10))

  0%|          | 0/100 [00:00<?, ?it/s]


----------- Epoch: 1 of 100 -----------

Training Results for epoch: 1 of 100
Training Loss: 0.8217929307877331


  1%|          | 1/100 [00:08<14:45,  8.95s/it]

Training Accuracy: 70.51810237203495
Validation Results for epoch: 1 of 100
Validation Loss: 0.8311205663313597
Validation Accuracy: 69.14141414141413

----------- Epoch: 2 of 100 -----------

Training Results for epoch: 2 of 100
Training Loss: 0.4917838474509859


  2%|▏         | 2/100 [00:16<13:33,  8.30s/it]

...

Training Accuracy: 97.99417394923013
Validation Results for epoch: 98 of 100
Validation Loss: 0.05477251511587635
Validation Accuracy: 97.74410774410775

----------- Epoch: 99 of 100 -----------

Training Results for epoch: 99 of 100
Training Loss: 0.06295398530110559
Training Accuracy: 97.60299625468166
Validation Results for epoch: 99 of 100
Validation Loss: 0.07084267874098943


 99%|█████████▉| 99/100 [16:14<00:08,  8.58s/it]

Validation Accuracy: 97.23905723905723

----------- Epoch: 100 of 100 -----------

Training Results for epoch: 100 of 100
Training Loss: 0.04926162058678624


100%|██████████| 100/100 [16:24<00:00,  9.84s/it]

Training Accuracy: 98.08780690803162
Validation Results for epoch: 100 of 100
Validation Loss: 0.05315767084655088
Validation Accuracy: 97.96296296296296


In [36]:
#Training using Tanh Activation function
nn2 = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="tanh")
nn2.fit(np.array(mnist_train_flat, dtype='float32'), nn2.one_hot_encode(mnist_train_labels.astype(int),10))

  0%|          | 0/100 [00:00<?, ?it/s]


----------- Epoch: 1 of 100 -----------

Training Results for epoch: 1 of 100
Training Loss: 0.6282535078075241


  1%|          | 1/100 [00:08<14:39,  8.89s/it]

Training Accuracy: 76.67707032875572
Validation Results for epoch: 1 of 100
Validation Loss: 0.6379690397581033
Validation Accuracy: 76.1952861952862

----------- Epoch: 2 of 100 -----------

Training Results for epoch: 2 of 100
Training Loss: 0.47898384682305867


  2%|▏         | 2/100 [00:16<13:32,  8.29s/it]

...

Validation Accuracy: 97.5925925925926

----------- Epoch: 99 of 100 -----------

Training Results for epoch: 99 of 100
Training Loss: 0.060030500681698885
Training Accuracy: 97.73616312942156
Validation Results for epoch: 99 of 100
Validation Loss: 0.05816814580099681


 99%|█████████▉| 99/100 [16:52<00:10, 10.24s/it]

Validation Accuracy: 97.71043771043772

----------- Epoch: 100 of 100 -----------

Training Results for epoch: 100 of 100
Training Loss: 0.07517487984032123


100%|██████████| 100/100 [17:04<00:00, 10.24s/it]

Training Accuracy: 97.23054515189347
Validation Results for epoch: 100 of 100
Validation Loss: 0.0713409377629896
Validation Accuracy: 97.22222222222221


In [37]:
#Training using Sigmoid Activation function
nn3 = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="sigmoid")
nn3.fit(np.array(mnist_train_flat, dtype='float32'), nn3.one_hot_encode(mnist_train_labels.astype(int),10))

  0%|          | 0/100 [00:00<?, ?it/s]


----------- Epoch: 1 of 100 -----------

Training Results for epoch: 1 of 100
Training Loss: 2.305696110137466


  1%|          | 1/100 [00:14<24:19, 14.74s/it]

Training Accuracy: 9.956304619225968
Validation Results for epoch: 1 of 100
Validation Loss: 2.307465263129674
Validation Accuracy: 10.0

----------- Epoch: 2 of 100 -----------

Training Results for epoch: 2 of 100
Training Loss: 2.304471219469107
Training Accuracy: 9.889721181856013
Validation Results for epoch: 2 of 100
Validation Loss: 2.3038857159099235


  2%|▏         | 2/100 [00:24<19:13, 11.77s/it]

...

Training Accuracy: 93.89513108614233
Validation Results for epoch: 98 of 100
Validation Loss: 0.17549885428689607
Validation Accuracy: 93.67003367003367

----------- Epoch: 99 of 100 -----------

Training Results for epoch: 99 of 100
Training Loss: 0.16763885590830238


 99%|█████████▉| 99/100 [15:05<00:07,  7.72s/it]

Training Accuracy: 94.06783187682063
Validation Results for epoch: 99 of 100
Validation Loss: 0.17792680864049418
Validation Accuracy: 93.85521885521885

----------- Epoch: 100 of 100 -----------

Training Results for epoch: 100 of 100
Training Loss: 0.1761787608792819
Training Accuracy: 93.60382854764877
Validation Results for epoch: 100 of 100
Validation Loss: 0.1770458594997357


100%|██████████| 100/100 [15:13<00:00,  9.13s/it]

Validation Accuracy: 93.58585858585859


In [38]:
#Training using Linear Activation. Please note that due to a high learning rate, the gradients explode at the 27th epoch
nn4 = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="linear")
nn4.fit(np.array(mnist_train_flat, dtype='float32'), nn4.one_hot_encode(mnist_train_labels.astype(int),10))

  0%|          | 0/100 [00:00<?, ?it/s]


----------- Epoch: 1 of 100 -----------

Training Results for epoch: 1 of 100
Training Loss: 0.8197277447091772


  1%|          | 1/100 [00:12<20:45, 12.58s/it]

Training Accuracy: 71.35871826883063
Validation Results for epoch: 1 of 100
Validation Loss: 0.8294841921730912
Validation Accuracy: 70.70707070707071

----------- Epoch: 2 of 100 -----------

Training Results for epoch: 2 of 100
Training Loss: 0.670848358089954


  2%|▏         | 2/100 [00:19<14:50,  9.08s/it]

...

Training Accuracy: 9.96046608406159
Validation Results for epoch: 98 of 100
Validation Loss: nan
Validation Accuracy: 10.37037037037037

----------- Epoch: 99 of 100 -----------

Training Results for epoch: 99 of 100
Training Loss: nan


 99%|█████████▉| 99/100 [10:37<00:06,  6.50s/it]

Training Accuracy: 9.987515605493133
Validation Results for epoch: 99 of 100
Validation Loss: nan
Validation Accuracy: 10.151515151515152

----------- Epoch: 100 of 100 -----------

Training Results for epoch: 100 of 100
Training Loss: nan


100%|██████████| 100/100 [10:43<00:00,  6.44s/it]

Training Accuracy: 10.004161464835622
Validation Results for epoch: 100 of 100
Validation Loss: nan
Validation Accuracy: 10.016835016835017


The learned weights and biases for each activation function after 50 epochs and 100 epochs have been saved in /checkpoints/activation-function-name

5 (c). Test saved models at 50 and 100 epochs on test set for all activations.

In [28]:
#load weights from the saved pickle files and report accuracies on the test dataset 
my_accuracies = pd.DataFrame(columns=['50_epochs', '100_epochs'], index=["sigmoid", "tanh", "relu", "linear"])
for a in ["sigmoid", "tanh", "relu", "linear"]:
    acc = []
    for epoch in ['50','100']:
        test_nn = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=5, batch_size=32, weight_init='normal', activation=a)
        weights_file = "checkpoints/"+a+"/"+epoch+"_"+a+"_weights.pkl"
        test_nn.weights = joblib.load(weights_file)
        bias_file = "checkpoints/"+a+"/"+epoch+"_"+a+"_bias.pkl"
        test_nn.bias = joblib.load(bias_file)
        scr = test_nn.score(np.array(mnist_test_flat, dtype='float32'), mnist_test_labels.astype(int))*100
        acc.append(scr)
    my_accuracies.loc[a] = acc

In [31]:
print("Accuracies on test set for my implementation:")
print(tabulate(my_accuracies, headers='keys', tablefmt='psql'))

Accuracies on test set for my implementation:
+---------+-------------+--------------+
|         |   50_epochs |   100_epochs |
|---------+-------------+--------------|
| sigmoid |     88.4667 |        88.8  |
| tanh    |     89.2833 |        88.45 |
| relu    |     89.7    |        89.45 |
| linear  |      9.95   |         9.95 |
+---------+-------------+--------------+


5(b). Training loss vs epoch curve and validation loss vs epoch curve for ReLU and Sigmoid activation function

In [20]:
def plot_loss_curve():
    sns.set()
    fig, axes = plt.subplots(1, 2, figsize=(15,7))#15 7
    plt.subplots_adjust(wspace=0.2, hspace=0.4)
    axes[0].set_title("ReLU")
    axes[1].set_title("Sigmoid")
    sns.lineplot(x=np.linspace(1,100,100), y=training_loss["relu"], ax=axes[0])
    sns.lineplot(x=np.linspace(1,100,100), y=validation_loss["relu"], ax=axes[0])
    sns.lineplot(x=np.linspace(1,100,100), y=training_loss["sigmoid"], ax=axes[1])
    sns.lineplot(x=np.linspace(1,100,100), y=validation_loss["sigmoid"], ax=axes[1])
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("Loss")
    axes[0].legend(loc='upper right', labels=['training', 'validation'])
    axes[1].legend(loc='upper right', labels=['training', 'validation'])
    fig.suptitle("Loss vs Epoch Curves")
    plt.savefig("loss_curves_relu_sigmoid.png")
    plt.show()

In [21]:
plot_loss_curve()

##### **Question 1: Part 6**

In [44]:
#run sklearn's MLP with ReLU as the loss function and plot losses for both.
nn1_ = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="relu")
nn1_.compare_with_mlp(training_loss["relu"], mnist_train_flat, mnist_train_labels.astype(int))

Iteration 1, loss = 0.54956344
Iteration 2, loss = 0.39268738
Iteration 3, loss = 0.35180555
Iteration 4, loss = 0.32548673
Iteration 5, loss = 0.30448772
Iteration 6, loss = 0.28825450
Iteration 7, loss = 0.27755963
Iteration 8, loss = 0.26499282
Iteration 9, loss = 0.25293950
Iteration 10, loss = 0.24534916
Iteration 11, loss = 0.23786828
Iteration 12, loss = 0.22874060
Iteration 13, loss = 0.22453215
Iteration 14, loss = 0.21619593
Iteration 15, loss = 0.20884032
Iteration 16, loss = 0.20166635
Iteration 17, loss = 0.19582943
Iteration 18, loss = 0.19350689
Iteration 19, loss = 0.18825211
Iteration 20, loss = 0.18463253
Iteration 21, loss = 0.18053355
Iteration 22, loss = 0.17362635
Iteration 23, loss = 0.16846729
Iteration 24, loss = 0.16503565
Iteration 25, loss = 0.16014472
Iteration 26, loss = 0.15701411
Iteration 27, loss = 0.15052729
Iteration 28, loss = 0.15247049
Iteration 29, loss = 0.15094679
Iteration 30, loss = 0.14705859
Iteration 31, loss = 0.14168726
Iteration 32, los

/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 51, loss = 0.09510056
Iteration 52, loss = 0.09332632
Iteration 53, loss = 0.09017744
Iteration 54, loss = 0.08752531
Iteration 55, loss = 0.08784727
Iteration 56, loss = 0.08944281
Iteration 57, loss = 0.08659817
Iteration 58, loss = 0.08881744
Iteration 59, loss = 0.08484203
Iteration 60, loss = 0.08044382
Iteration 61, loss = 0.08373300
Iteration 62, loss = 0.09186337
Iteration 63, loss = 0.08279339
Iteration 64, loss = 0.07727548
Iteration 65, loss = 0.07703788
Iteration 66, loss = 0.07957713
Iteration 67, loss = 0.07602993
Iteration 68, loss = 0.07929532
Iteration 69, loss = 0.06883541
Iteration 70, loss = 0.06996957
Iteration 71, loss = 0.07174112
Iteration 72, loss = 0.07314944
Iteration 73, loss = 0.07341223
Iteration 74, loss = 0.06485513
Iteration 75, loss = 0.06167944
Iteration 76, loss = 0.07091131
Iteration 77, loss = 0.06652484
Iteration 78, loss = 0.06786373
Iteration 79, loss = 0.05557649
Iteration 80, loss = 0.05690579
Iteration 81, loss = 0.05954619
Iteratio

In [45]:
#run sklearn's MLP with tanh as the loss function and plot losses for both.
nn2_ = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="tanh")
nn2_.compare_with_mlp(training_loss["tanh"], mnist_train_flat, mnist_train_labels.astype(int))

Iteration 1, loss = 0.51471381
Iteration 2, loss = 0.39012565
Iteration 3, loss = 0.35265138
Iteration 4, loss = 0.32906033
Iteration 5, loss = 0.30909565
Iteration 6, loss = 0.29511740
Iteration 7, loss = 0.28192320
Iteration 8, loss = 0.27023572
Iteration 9, loss = 0.26000107
Iteration 10, loss = 0.25062298
Iteration 11, loss = 0.24221577
Iteration 12, loss = 0.23350708
Iteration 13, loss = 0.22560617
Iteration 14, loss = 0.21748704
Iteration 15, loss = 0.21119399
Iteration 16, loss = 0.20439609
Iteration 17, loss = 0.19863052
Iteration 18, loss = 0.19401454
Iteration 19, loss = 0.18857751
Iteration 20, loss = 0.18224309
Iteration 21, loss = 0.17595456
Iteration 22, loss = 0.17128181
Iteration 23, loss = 0.16842952
Iteration 24, loss = 0.16239848
Iteration 25, loss = 0.15823108
Iteration 26, loss = 0.15551763
Iteration 27, loss = 0.14976591
Iteration 28, loss = 0.14495453
Iteration 29, loss = 0.14051096
Iteration 30, loss = 0.13863305
Iteration 31, loss = 0.13528029
Iteration 32, los

/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 51, loss = 0.08713591
Iteration 52, loss = 0.08374984
Iteration 53, loss = 0.08346095
Iteration 54, loss = 0.07820357
Iteration 55, loss = 0.07559019
Iteration 56, loss = 0.07682625
Iteration 57, loss = 0.07147350
Iteration 58, loss = 0.07782127
Iteration 59, loss = 0.06840677
Iteration 60, loss = 0.06892909
Iteration 61, loss = 0.06895629
Iteration 62, loss = 0.06917370
Iteration 63, loss = 0.07412648
Iteration 64, loss = 0.07033283
Iteration 65, loss = 0.05662873
Iteration 66, loss = 0.05580843
Iteration 67, loss = 0.05998552
Iteration 68, loss = 0.06369082
Iteration 69, loss = 0.05625851
Iteration 70, loss = 0.06043907
Iteration 71, loss = 0.06578914
Iteration 72, loss = 0.05879139
Iteration 73, loss = 0.05018764
Iteration 74, loss = 0.05452103
Iteration 75, loss = 0.05777708
Iteration 76, loss = 0.05448226
Iteration 77, loss = 0.05245753
Iteration 78, loss = 0.06224584
Iteration 79, loss = 0.05127826
Iteration 80, loss = 0.05210366
Iteration 81, loss = 0.05600223
Iteratio

In [46]:
#run sklearn's MLP with sigmoid as the loss function and plot losses for both.
nn3_ = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="sigmoid")
nn3_.compare_with_mlp(training_loss["sigmoid"], mnist_train_flat, mnist_train_labels.astype(int))

Iteration 1, loss = 2.02367717
Iteration 2, loss = 0.95720552
Iteration 3, loss = 0.74486950
Iteration 4, loss = 0.64532120
Iteration 5, loss = 0.57201175
Iteration 6, loss = 0.52005666
Iteration 7, loss = 0.48428663
Iteration 8, loss = 0.45897302
Iteration 9, loss = 0.43928874
Iteration 10, loss = 0.42327233
Iteration 11, loss = 0.40899844
Iteration 12, loss = 0.39609271
Iteration 13, loss = 0.38499294
Iteration 14, loss = 0.37518376
Iteration 15, loss = 0.36618427
Iteration 16, loss = 0.35984990
Iteration 17, loss = 0.35189751
Iteration 18, loss = 0.34644228
Iteration 19, loss = 0.34013237
Iteration 20, loss = 0.33245079
Iteration 21, loss = 0.32819861
Iteration 22, loss = 0.32184883
Iteration 23, loss = 0.31775343
Iteration 24, loss = 0.31295533
Iteration 25, loss = 0.30934649
Iteration 26, loss = 0.30446354
Iteration 27, loss = 0.29972459
Iteration 28, loss = 0.29678959
Iteration 29, loss = 0.29238674
Iteration 30, loss = 0.28929531
Iteration 31, loss = 0.28534028
Iteration 32, los

/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 51, loss = 0.22609717
Iteration 52, loss = 0.22432152
Iteration 53, loss = 0.22201128
Iteration 54, loss = 0.21788792
Iteration 55, loss = 0.21718861
Iteration 56, loss = 0.21472701
Iteration 57, loss = 0.21216861
Iteration 58, loss = 0.20918891
Iteration 59, loss = 0.20893843
Iteration 60, loss = 0.20486352
Iteration 61, loss = 0.20328249
Iteration 62, loss = 0.20137630
Iteration 63, loss = 0.19995307
Iteration 64, loss = 0.19628347
Iteration 65, loss = 0.19503197
Iteration 66, loss = 0.19313242
Iteration 67, loss = 0.19057666
Iteration 68, loss = 0.18959462
Iteration 69, loss = 0.18700780
Iteration 70, loss = 0.18486090
Iteration 71, loss = 0.18423520
Iteration 72, loss = 0.18156283
Iteration 73, loss = 0.18189041
Iteration 74, loss = 0.17808105
Iteration 75, loss = 0.17534483
Iteration 76, loss = 0.17376319
Iteration 77, loss = 0.17161380
Iteration 78, loss = 0.17011892
Iteration 79, loss = 0.16819045
Iteration 80, loss = 0.16750199
Iteration 81, loss = 0.16480673
Iteratio

In [47]:
#run sklearn's MLP with linear as the loss function and plot losses for both.
nn4_ = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=100, batch_size=32, weight_init='normal', activation="linear")
nn4_.compare_with_mlp(training_loss["linear"], mnist_train_flat, mnist_train_labels.astype(int))

/usr/local/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/usr/local/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Iteration 1, loss = nan
Iteration 2, loss = nan
Iteration 3, loss = nan
Iteration 4, loss = nan
Iteration 5, loss = nan
Iteration 6, loss = nan
Iteration 7, loss = nan
Iteration 8, loss = nan
Iteration 9, loss = nan
Iteration 10, loss = nan
Iteration 11, loss = nan
Iteration 12, loss = nan
Iteration 13, loss = nan
Iteration 14, loss = nan
Iteration 15, loss = nan
Iteration 16, loss = nan
Iteration 17, loss = nan
Iteration 18, loss = nan
Iteration 19, loss = nan
Iteration 20, loss = nan
Iteration 21, loss = nan
Iteration 22, loss = nan
Iteration 23, loss = nan
Iteration 24, loss = nan
Iteration 25, loss = nan
Iteration 26, loss = nan
Iteration 27, loss = nan
Iteration 28, loss = nan
Iteration 29, loss = nan
Iteration 30, loss = nan
Iteration 31, loss = nan
Iteration 32, loss = nan
Iteration 33, loss = nan
Iteration 34, loss = nan
Iteration 35, loss = nan
Iteration 36, loss = nan
Iteration 37, loss = nan
Iteration 38, loss = nan
Iteration 39, loss = nan
Iteration 40, loss = nan
Iteration

/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 51, loss = nan
Iteration 52, loss = nan
Iteration 53, loss = nan
Iteration 54, loss = nan
Iteration 55, loss = nan
Iteration 56, loss = nan
Iteration 57, loss = nan
Iteration 58, loss = nan
Iteration 59, loss = nan
Iteration 60, loss = nan
Iteration 61, loss = nan
Iteration 62, loss = nan
Iteration 63, loss = nan
Iteration 64, loss = nan
Iteration 65, loss = nan
Iteration 66, loss = nan
Iteration 67, loss = nan
Iteration 68, loss = nan
Iteration 69, loss = nan
Iteration 70, loss = nan
Iteration 71, loss = nan
Iteration 72, loss = nan
Iteration 73, loss = nan
Iteration 74, loss = nan
Iteration 75, loss = nan
Iteration 76, loss = nan
Iteration 77, loss = nan
Iteration 78, loss = nan
Iteration 79, loss = nan
Iteration 80, loss = nan
Iteration 81, loss = nan
Iteration 82, loss = nan
Iteration 83, loss = nan
Iteration 84, loss = nan
Iteration 85, loss = nan
Iteration 86, loss = nan
Iteration 87, loss = nan
Iteration 88, loss = nan
Iteration 89, loss = nan
Iteration 90, loss = nan


In [10]:
validation_loss = joblib.load("validation_loss.pkl")
training_loss = joblib.load("training_loss.pkl")

Plots obtained after training Sklearn's MLP for each of the given activations are:

* ReLU activation

![ReLU-Loss-Comparison](relu_comparison.png)

* Sigmoid activation

![Sigmoid-Loss-Comparison](sigmoid_comparison.png)

* Tanh activation

![Tanh-Loss-Comparison](tanh_comparison.png)

* Linear activation

![Linear-Loss-Comparison](linear_comparison.png)

In [56]:
#load weights from the saved pickle files for sklearn and my implementation for 50 and 100 epochs and create a dataframe of accuracy on test set
accuracies = pd.DataFrame(columns=["sklearn_50_epochs", "mine_50_epochs", "sklearn_100_epochs", "mine_100_epochs"], index=["sigmoid", "tanh", "relu", "linear"])
best_accuray = 0
best_model = None
model_type = ''
best_epoch = ''
for a in ["sigmoid", "tanh", "relu", "linear"]:
    acc = []
    for epoch in ['50','100']:
        test_nn = MyNeuralNetwork(n_layers=3, layer_sizes=[256, 128, 64], learning_rate=0.1, num_epochs=5, batch_size=32, weight_init='normal', activation=a)
        weights_file = "checkpoints/"+a+"/"+epoch+"_"+a+"_weights.pkl"
        test_nn.weights = joblib.load(weights_file)
        bias_file = "checkpoints/"+a+"/"+epoch+"_"+a+"_bias.pkl"
        test_nn.bias = joblib.load(bias_file)
        sklearn_path = "checkpoints/sklearn/"+epoch+"_"+a+"_model.pkl"
        sklearn_model = joblib.load(sklearn_path)
        acc1 = sklearn_model.score(np.array(mnist_test_flat, dtype='float32'), mnist_test_labels.astype(int))*100
        acc2 = test_nn.score(np.array(mnist_test_flat, dtype='float32'), mnist_test_labels.astype(int))*100
        acc.append(acc1)
        acc.append(acc2)
        if (acc1>best_accuray):
            best_accuray = acc1
            best_model = sklearn_model
            model_type = 'sklearn'
            best_epoch = epoch
        if (acc2>best_accuray):
            best_accuray = acc2
            best_model = test_nn
            model_type = 'my'
            best_epoch = epoch
    accuracies.loc[a] = acc

In [57]:
#accuracy of all loaded models:
print("Accuracies on test set for sklearn and my implementation:")
print(tabulate(accuracies, headers='keys', tablefmt='psql'))

Accuracies on test set for sklearn and my implementation:
+---------+---------------------+------------------+----------------------+-------------------+
|         |   sklearn_50_epochs |   mine_50_epochs |   sklearn_100_epochs |   mine_100_epochs |
|---------+---------------------+------------------+----------------------+-------------------|
| sigmoid |             88.3167 |          88.4667 |              87.9833 |             88.8  |
| tanh    |             88.05   |          89.2833 |              87.5667 |             88.45 |
| relu    |             88.7333 |          89.7    |              88.7167 |             89.45 |
| linear  |              9.95   |           9.95   |               9.95   |              9.95 |
+---------+---------------------+------------------+----------------------+-------------------+


In [59]:
#best overall model in terms of accuracy
print("Best model in terms of accuracy is "+model_type+ " MLP with "+best_model.activation+" as the loss function after "+ best_epoch+" epochs")
print("Best accuracy obtained: "+str(best_accuray)+"%")

Best model in terms of accuracy is my MLP with relu as the loss function after 50 epochs
Best accuracy obtained: 89.7%
